### 梯度下降法的向量化和数据的标准化
#### 1.对梯度多向量化处理
- 对计算公式向量化后，使计算更简单,向量化过程:
![计算公式向量化](https://images2018.cnblogs.com/blog/1355387/201806/1355387-20180605095313063-1781537591.png)

- 最终的向量化结果:

![](https://images2018.cnblogs.com/blog/1355387/201806/1355387-20180605100215696-1890972506.png)
- 其中,XbT == Xb . T：是 Xb 的转置矩阵;

- 改变梯度的计算方式,(更改函数数 dJ()):

In [21]:
# 原来梯度计算函数
# def dJ(theta, X_b, y):
# 
#     result = np.empty(len(theta))  # 构建大小为len(theta)的空列表
#     result[0] = np.sum(X_b.dot(theta) - y)  # 先处理theta_0的导数
#     
#     # 对之后的所有theta进行求导,除了第一列theta_0
#     for i in range(1, len(theta)):
#         result[i] = (X_b.dot(theta) - y).dot(X_b[:, i])
#         
#     return 2 * result / len(X_b)

# 更改为
def dJ(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2 / len(X_b)

#### 2. 实例

- 构造及处理数据

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from beeprint import pp  # 打印库,更清晰得打印日志

# 波士顿房产数据
boston_data = datasets.load_boston()
X = boston_data.data
y = boston_data.target

# 去掉最高房价数据
X = X[y < 50]
y = y[y < 50]

# 分割数据
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100)

- 使用正规方程解优化损失函数

In [3]:
from LinearRegression import LinearRegression

estimator_reg1 = LinearRegression()

%time estimator_reg1.fit_normal(X_train, y_train)
estimator_reg1.score(X_test, y_test)

Wall time: 998 µs


0.8223161067005949

- 使用梯度下降法优化损失函数

In [24]:
estimator_reg2 = LinearRegression()

%time estimator_reg2.fit_gd(X_train, y_train)
# estimator_reg2.score(X_test, y_test)

estimator_reg2.score(X_test, y_test)

f:\language\python\lib\site-packages\numpy\core\fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
F:\WorkSpaces\MLlearning-notes\ml\05-Gradient-Descent\03-Vectorize-Gradient-Descent\LinearRegression.py:48: RuntimeWarning: overflow encountered in square
  return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
F:\WorkSpaces\MLlearning-notes\ml\05-Gradient-Descent\03-Vectorize-Gradient-Descent\LinearRegression.py:68: RuntimeWarning: invalid value encountered in double_scalars
  
F:\WorkSpaces\MLlearning-notes\ml\05-Gradient-Descent\03-Vectorize-Gradient-Descent\LinearRegression.py:57: RuntimeWarning: overflow encountered in multiply
  theta = initial__theta


Wall time: 1.06 s


nan

输出结果有：RuntimeWarning: overflow

表示某些值太大了，超出Python限定的范围了，所以输出结果为‘nan’

问题一：没有得到准确度，说明优化时损失函数没有收敛

分析：数据集中的数据值的大小差异较大，使用默认的 eta 求得的梯度非常大，使得梯度下降法的结果不收敛

方案：修改 eta 值

- 修改 eta 值,再次优化

In [25]:
estimator_reg2.fit_gd(X_train, y_train, eta=0.000001)  # 默认值为0.01，这次我们缩小10000倍
estimator_reg2.score(X_test, y_test)

0.2655087616938242

问题二：准确度太小（与正规方程优化结果比较而言）；

分析： eta 值太小，根据设定的循环次数没能找到损失函数的最小值；

方案：修改循环次数

- 修改循环次数，再次优化

In [26]:
%time estimator_reg2.fit_gd(X_train, y_train, eta=0.000001, n_iters=10e6)
estimator_reg2.score(X_test, y_test)

Wall time: 16min 59s


0.7955785517387927

问题三：耗时较长，如果循环次数过大，循环耗时比较大，另外，eta 太小也会耗时较大，因为步长太小，需要经过多次梯度下降才有可能取最小值

分析（1）：之所以出现这种现象，是因为整体的数据集的数值不在同一个规模上，也就是大小相差太大；

分析（2）：由于有 eta 这个变量，如果最终数据集的数据不在一个维度上（大小相差很大）将会影响梯度的结果，
梯度的结果 乘以 eta 是损失函数真正每次变化的步长，则步长有可能或者太大或者太小。

如果步长太大会导致损结果不收敛，如果步长太小会导致搜索过程太慢；

方案：对数据集做归一化处理
 
注意：**用正规方程解线性回归的损失函数时，不需要对数据集归一化处理。**

因为正规方程中的模型的求解过程，整体变成了一个公式的计算，在公式计算过程中牵涉到的搜索的过程比较少

- 对数据做归一化处理，再次使用梯度下降法

In [27]:
from sklearn.preprocessing import StandardScaler

# 标准归一化处理
stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_test_stand = stand_scaler.fit_transform(X_test)


# 再次使用梯度下降法

estimator_reg3 = LinearRegression()
%time estimator_reg3.fit_gd(X_train_stand, y_train)

estimator_reg3.score(X_test_stand, y_test)  # 

Wall time: 495 ms


0.8170765550957655

观察得分，与正规化方程所得结果相近

优点：没有更改默认的 eta 和循环次数 n_iters，fit 速度很快

仔细观察数据，在刚开始使用正规方程解优化损失函数时，耗时998 µs，得分：0.8223161067005949

之后用梯度下降法优化损失函数时，耗时495ms，得分0.8170765550957655

比较之后，发现正规方程解比梯度下降好用啊，为啥还要学习梯度下降法？

因为正规方程处理的是 m 行 n 列的矩阵，对矩阵进行非常多的乘法运算，如果矩阵比较大时，用正规方程法优化耗时就会较多

在本案例中，数据量较小，看不出梯度下降的优势

- 梯度下降法比正规化方程优化法的优势实践

In [4]:
# 构造数据
m = 1000  # 1000行数据
n = 5000  # 5000个特征，在实际工作中，50000个特征可能都算小的

big_x = np.random.normal(size=(m,n))  # 正态分布 m*n，不需要数据归一化了
true_theta = np.random.uniform(0.0, 100., size=n+1)  # 模拟theta系数 uniform从均匀分布中抽取样本。样本均匀分布在半开区间``[low，high）'' ,大小为n+1
big_y = big_x.dot(true_theta[1:]) + true_theta[0] + np.random.normal(0., 10, size=m)  # 自定义线性回归方程，数据量和x保持一致

In [5]:
# 正规方程优化耗时
big_esti1 = LinearRegression()
%time big_esti1.fit_normal(big_x, big_y)

Wall time: 3.79 s


LinearRegression()

In [6]:
big_esti2 = LinearRegression()
%time big_esti2.fit_gd(big_x, big_y)


Wall time: 2.73 s


LinearRegression()

优点：使用梯度下降法优化，耗时更少

原因：正规方程处理的是 m 行 n 列的矩阵，对矩阵进行非常多的乘法运算，如果矩阵比较大时，用正规方程法优化耗时较多


但是这种梯度下降法也有缺点：

本次举例中，样本数量（m）小于特征量（n），目前在使用梯度下降法计算梯度时，要让每一个样本都参与计算，如果样本量比较大时，计算梯度也比较慢；

方案：使用随机梯度下降法